In [101]:
import numpy as np
import pandas as pd
from sklearn.impute import (SimpleImputer, KNNImputer)
from sklearn.feature_selection import SelectKBest

In [244]:

df_train_features = pd.read_csv('./train_features.csv')
df_train_labels   = pd.read_csv('./train_labels.csv')
df_test_features  = pd.read_csv('./test_features.csv')
X = df_train_features.values
label=df_train_labels.values
X_test = df_test_features.values
X.shape

(227940, 37)

In [245]:
idf=df_train_features.iloc[:,0]
ifl=df_train_labels.iloc[:,0]
idt=df_test_features.iloc[:,0]
idf_unique = pd.unique(idf)
idt_unique=pd.unique(idt)

In [246]:
def preprocessing(X, n_neighbors=75, method='KNN'):

    X_std= np.nanstd(X, axis=0, keepdims=True)
    X_mean= np.nanmean(X, axis=0, keepdims=True)
    X_norm= (X- X_mean)/X_std

    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights='distance') if method =='KNN'\
        else SimpleImputer (missing_values=np.nan, strategy='median')

    X_norm_fixed= imputer.fit_transform(X_norm)

    return X_norm_fixed

In [247]:
X_norm_fixed = preprocessing(X, method = 'median')
X_test_fixed = preprocessing(X_test, method = 'median')
print(X_norm_fixed)
pid_idx = np.where(X[:,0]==10)
print(pid_idx)

[[-1.7250923  -0.85121278 -1.70642596 ... -0.28285627  0.84344393
  -0.50054209]
 [-1.7250923  -0.63917285 -1.70642596 ... -0.28285627  0.113008
  -0.50054209]
 [-1.7250923  -0.42713292 -1.70642596 ... -0.28285627 -0.53149428
   0.03721993]
 ...
 [-0.63263872  0.63306673  1.39353544 ... -0.28285627  0.67157665
   0.03721993]
 [-0.63263872  0.84510666  1.39353544 ... -0.28285627  0.113008
   0.03721993]
 [-0.63263872  1.05714659  1.39353544 ... -0.28285627  0.24190846
   0.03721993]]
(array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], dtype=int64),)


In [248]:
def get_patient_data(pid):
    pid_idx = np.where(X[:,0]==pid)
    return X_norm_fixed[pid_idx, :].reshape([12,37])

def get_patient_data_test(pid_index):
    pid_idx = np.where(X_test[:,0]==pid_index)
    return X_test_fixed[pid_idx, :].reshape([12,37])

In [124]:
X_1=get_patient_data(10)
X_1.shape

(12, 37)

In [121]:
X_norm_fixed[12,:]

array([-1.7241089 , -1.27529265,  0.54256564,  0.01498224, -0.30311906,
       -0.30926529, -0.31286483,  0.16895924, -0.06179241,  0.09353697,
        0.05707343, -0.03058222, -0.22179091, -0.1535099 , -0.13268913,
       -0.31388775, -0.12494276, -0.23059474, -0.00822398, -0.15041668,
        0.3664487 , -0.21389641, -0.1285392 , -0.00948796, -0.078682  ,
       -0.14473443,  0.36943647, -0.21013043,  0.12079291, -0.39083107,
        0.12505594, -0.06638969, -0.0862856 , -0.25924228, -0.28285627,
       -0.14479291,  0.03721993])

Sub-task 1
<font color = blue>
    

In [249]:
#--training data preprocessing
x_imputed = np.zeros((int(df_train_features.shape[0]/12), 35))

for pid_idx in range(len(idf_unique)):
    # get training data for a patient (dim 12 x d)
    x_patient = get_patient_data(idf_unique[pid_idx])
    x_patient_imputed = np.array([0.]*35)
    
    # deal with column seperately
    for col in range(len(x_patient[1, 2:])):

         x_patient_imputed[col]=np.mean(x_patient, axis = 0)[col]
         
    # Concatenate imputed data for every patient
    
    x_imputed[pid_idx, :] = x_patient_imputed
    
print(x_imputed)


[[-1.7250923   0.31500683 -1.70642596 ... -0.42163303 -0.25924228
  -0.28285627]
 [-1.7241089  -0.10907303  0.54256564 ... -0.30355295 -0.24639742
  -0.2833198 ]
 [-1.71427485  0.1029669   0.36021497 ...  1.39207704 -0.25924228
  -0.28285627]
 ...
 [-0.63296652  1.16316656 -0.55153838 ...  0.79223022 -0.25924228
  -0.28285627]
 [-0.63274798 -0.10907303  1.63666967 ...  0.66470373 -0.25924228
  -0.27388363]
 [-0.63263872 -0.10907303  1.39353544 ... -0.33661537 -0.25924228
  -0.28285627]]


In [179]:
# test data preprocessing
x_test_imputed = np.zeros((int(df_test_features.shape[0]/12), 35))

for pid_idx in range(len(idt_unique)):
   
    x_test_patient = get_patient_data_test(idt_unique[pid_idx])
    x_test_patient_imputed = np.array([0.]*35)
    
    # deal with column seperately
    for col in range(len(x_test_patient[1, 2:])):
      
        x_test_patient_imputed[col]=np.mean(x_test_patient, axis = 0)[col]
            
    # Concatenate imputed data for every patient
    x_test_imputed[pid_idx, :] = x_test_patient_imputed
    
print(x_test_imputed)

[[-1.74229257 -0.11283712 -1.41719456 ... -0.02989306  0.91381053
  -0.2958202 ]
 [-0.64566007 -0.11283712 -0.0094439  ...  1.2601508  -0.26411003
  -0.2958202 ]
 [-0.64544076 -0.11283712  0.6026216  ...  0.33056037 -0.26411003
  -0.2958202 ]
 ...
 [-0.64664694  0.33801295 -0.68271596 ... -0.14846327 -0.26411003
  -0.2958202 ]
 [-0.64642763  0.11258791 -0.62150941 ...  0.89495456 -0.2514714
  -0.2958202 ]
 [-0.64609868 -0.11283712 -0.31547666 ...  0.117134   -0.26411003
  -0.2958202 ]]


In [180]:
x_imputed.shape

(18995, 35)

In [250]:
label_2=df_train_labels.iloc[:,11]
label_2

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
18990    0.0
18991    0.0
18992    0.0
18993    0.0
18994    0.0
Name: LABEL_Sepsis, Length: 18995, dtype: float64

Subtask-1 and 2

In [274]:
Labels=['pid','LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2','LABEL_Sepsis','LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
result = pd.DataFrame(np.array([[0]*16]), columns=Labels)

In [221]:
y_train.shape

(18995, 11)

In [261]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import (GridSearchCV,
    cross_val_score, KFold)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import (HistGradientBoostingClassifier,RandomForestClassifier)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics

In [242]:



for i, label in enumerate(task12_labels):
    pipeline = make_pipeline(
                        StandardScaler(),
                        RandomForestClassifier(min_samples_leaf=20, class_weight='balanced', n_estimators=100))
    scores = cross_val_score(pipeline, x_imputed, y_train[:, i],
                                cv=5,
                                scoring='roc_auc')
    print("Cross-validation score is {score:.3f},"
          " standard deviation is {err:.3f}"
          .format(score = scores.mean(), err = scores.std()))

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [252]:
l=label[:,11]
print(l)
df_train_labels.iloc[:,11]

[0. 0. 0. ... 0. 0. 0.]


0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
18990    0.0
18991    0.0
18992    0.0
18993    0.0
18994    0.0
Name: LABEL_Sepsis, Length: 18995, dtype: float64

In [257]:
from tqdm import tqdm

In [260]:
# Fit the Random Forest on the whole training set
y_test=np.zeros((int(df_test_features.shape[0]/12), 11))
for i in tqdm(range(12)):
    y_train = label[:, i]

    c=RandomForestClassifier(min_samples_leaf=20, class_weight='balanced', n_estimators=100)


    brm_clf = OneVsRestClassifier(c)
    c.fit(x_imputed, y_train)

    # Predict the Labels for the test features

    y_test[:,i]=c.predict_proba(x_test_imputed)


  0%|          | 0/12 [00:22<?, ?it/s]


KeyboardInterrupt: 

In [255]:
y_tot.shape

(18995, 12)

In [271]:
x_tot = x_imputed
y_tot = label[:,1:12]

c=RandomForestClassifier(min_samples_leaf=20, class_weight='balanced', n_estimators=100)
# c.fit(x_tot,y_tot)

brm_clf = OneVsRestClassifier(c)
brm_clf.fit(x_tot, y_tot)

# Predict the Labels for the test features
x_test_tasksA_B = x_test_imputed
y_test2=brm_clf.predict_proba(x_test_tasksA_B)

In [272]:
y_test2.shape

(12664, 11)

In [276]:
TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2','LABEL_Sepsis']
i=0
for pid in idt_unique:
    result.loc[i,'pid']=pid
    result.loc[i,TESTS]=y_test2[i,:]
    i+=1

In [286]:

import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold
# from sklearn.svm import SVC
from sklearn.linear_model import BayesianRidge
from sklearn.impute import (SimpleImputer, KNNImputer)
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
# from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.multiclass import OneVsRestClassifier
scaler = MinMaxScaler()

In [287]:
def taskC_regressor(alpha_):
    c=Ridge(alpha=alpha_)

    # 10 fold cross-validation to choose hyperparameters
    num_fold = 10
    kf = KFold(n_splits=num_fold)
    score = 0
    for train_index, val_index in kf.split(x_imputed): # split the data into validation and training sets
        x_train = x_imputed[train_index]
        x_train_scaled = scaler.fit_transform(x_train)
        x_val = x_imputed[val_index]
        x_val_scaled = scaler.transform(x_val)
        
        y_train = y_[train_index, 11:15]
        y_val = y_[val_index, 11:15]
 
        c.fit(x_train_scaled, y_train) # apply model to training data
        y_est = np.transpose(c.predict(x_val_scaled)) # predict labels using validation data
        
        score += np.mean([0.5 + 0.5*np.maximum(0, r2_score(y_val[:,k],y_est[k,:])) for k in range(4)])
        print(np.mean([0.5 + 0.5*np.maximum(0, r2_score(y_val[:,k],y_est[k,:])) for k in range(4)]))

    # EVALUATION of model
    # Calculate average auroc score for all 10 folds
    avg_score = score/num_fold

    return avg_score

alpha_values = np.logspace(-3, 3, num=50, base=10.) #50 values between 10-3 to 10+3 
print (alpha_values)
avg_scores = [taskC_regressor(alpha) for alpha in alpha_values]
print(avg_scores)

# find alpha that has the highest score
best_alpha = alpha_values[np.argmax(avg_scores)]
print(best_alpha)

[1.00000000e-03 1.32571137e-03 1.75751062e-03 2.32995181e-03
 3.08884360e-03 4.09491506e-03 5.42867544e-03 7.19685673e-03
 9.54095476e-03 1.26485522e-02 1.67683294e-02 2.22299648e-02
 2.94705170e-02 3.90693994e-02 5.17947468e-02 6.86648845e-02
 9.10298178e-02 1.20679264e-01 1.59985872e-01 2.12095089e-01
 2.81176870e-01 3.72759372e-01 4.94171336e-01 6.55128557e-01
 8.68511374e-01 1.15139540e+00 1.52641797e+00 2.02358965e+00
 2.68269580e+00 3.55648031e+00 4.71486636e+00 6.25055193e+00
 8.28642773e+00 1.09854114e+01 1.45634848e+01 1.93069773e+01
 2.55954792e+01 3.39322177e+01 4.49843267e+01 5.96362332e+01
 7.90604321e+01 1.04811313e+02 1.38949549e+02 1.84206997e+02
 2.44205309e+02 3.23745754e+02 4.29193426e+02 5.68986603e+02
 7.54312006e+02 1.00000000e+03]
0.6692805030092729
0.6723505895275455
0.6719851175767041
0.6633323326024833
0.6637998771285136
0.6608466789162105
0.6661286776524615
0.5
0.6640950124431031
0.662069623992215
0.6692805963978907
0.6723523793111638
0.6719852095392884
0.663

0.6253223173877545
0.6184099226248166
0.6215119549498735
0.6223996613283945
0.6124571361856365
0.6155728471975349
0.6125592583716415
0.6139239105332188
0.5
0.6138295441697257
0.6151333511780208
0.606529403581821
0.6092455558733483
0.6102589996940568
0.6013158783011776
0.6039949993987742
0.6024102972312608
0.6024066717751122
0.5
0.6027383984607604
0.6038941732837195
0.5940799560299439
0.5964217101598106
0.5974851310515521
0.5896080976106338
0.5918346437482953
0.5916007034198568
0.5903553054645698
0.5
0.5910667403317137
0.592003991993928
0.581536371821437
0.5835292209834766
0.5845777045995864
0.5777748575639592
0.5795682327268311
0.580477563307892
0.5782294801936856
0.5
0.5792393621613064
0.5799302059556535
0.5693666466331674
0.5710471009325413
0.572030946871401
0.5662606956096381
0.5676638374547129
0.5694255457597125
0.5664833852796733
0.5
0.5676897951838973
0.5681434478194047
0.5579729019456195
0.5593840502092451
0.5602702687010679
0.5554538231197541
0.5565216443588277
0.55881918099087

In [288]:
x_tot_scaled = scaler.fit_transform(x_imputed)
y_tot = label[:, 11:15]

c=Ridge(alpha=best_alpha)
c.fit(x_tot_scaled, y_tot)

x_test_scaled=scaler.transform(x_test_imputed)
y_test3=(c.predict(x_test_scaled))

In [291]:
i=0
for pid in idt_unique:
    result.loc[i,'pid']=int(pid)
    result.loc[i,['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']]=y_test3[i, :]
    i+=1

In [293]:
# Save the output (predicted labels for the test features)
# output.to_csv('prediction2.csv', index=False, float_format='%.3f')
compression_opts = dict(method='zip', archive_name='prediction.csv')
result.to_csv('prediction.zip', index=False, float_format='%.3f', compression=compression_opts)